In [1]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import re
import os, sys

In [2]:
cwd = os.getcwd()
#print(cwd)
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
#print(parent_dir)
os.chdir(parent_dir)
print(os.getcwd())

from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.games import print_model as pm
from SBMLLint.games.som import SOM

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [49]:
# som.py was built using model #17, is being tested by model #18
# use reaction 'SHMTr' to create SOM to reduce reaction 'GARFT'
num = 17
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
pm.print_model(model)

<Model Hoefnagel2002_PyruvateBranches "Hoefnagel2002_PyruvateBranches">
R1: ADP + NAD + halfglucose -> ATP + NADH + pyruvate;
R2: NADH + pyruvate -> NAD + lactate;
R3: NAD + pyruvate + CoA -> NADH + AcCoA;
R4: AcCoA + PO4 -> CoA + AcP;
R5: ADP + AcP -> ATP + Ac;
R6: NADH + AcCoA -> NAD + CoA + AcO;
R7: NADH + AcO -> NAD + EtOH;
R8: pyruvate -> AcLac;
R9: AcLac -> AcetoinIn;
R10: AcetoinIn -> AcetoinOut;
R11: NADH + AcetoinIn -> NAD + Butanediol;
R12: ATP -> ADP;
R13: NADH + O2 -> NAD;
R14: AcLac -> AcetoinIn;


In [4]:
import unittest

In [5]:
UNIUNI = 'SHMTr'
MULTIMULTI = 'GARFT'
TEST_MOLECULE = 'CH2FH4'

In [6]:
simple = SimpleSBML(cn.TEST_FILE3)
sbml_reaction1 = simple._model.getReaction(UNIUNI)
sbml_reaction2 = simple._model.getReaction(MULTIMULTI)

In [7]:
Reaction.reactions = []
Reaction(sbml_reaction1)
Reaction(sbml_reaction2)
print(Reaction.reactions)

[CH2FH4 -> FH4, CHOFH4 + GAR -> FGAR + FH4]


In [8]:
reaction1 = Reaction.reactions[0]
reaction2 = Reaction.reactions[1]

In [9]:
print(Molecule.molecules)

[CH2FH4, FH4, CHOFH4, GAR, FGAR]


In [10]:
SOM.initialize(Molecule.molecules)

In [11]:
class TestSOM(unittest.TestCase):

    def setUp(self):
        self.simple = SimpleSBML(cn.TEST_FILE3)
        Reaction(simple._model.getReaction(UNIUNI))
        Reaction(simple._model.getReaction(MULTIMULTI))
        self.reaction1 = Reaction.reactions[0]
        self.reaction2 = Reaction.reactions[1]
        self.molecules = Molecule.molecules
        SOM.soms = []

    def testMakeId(self):
         SOM.initialize(self.molecules)
         self.assertEqual(len(SOM.soms), len(self.molecules))

In [12]:
# test Make Id
SOM.initialize(Molecule.molecules)
print(len(SOM.soms) == len(Molecule.molecules))

for som in SOM.soms:
    print(som.molecules.intersection(Molecule.molecules))

True
{CH2FH4}
{FH4}
{CHOFH4}
{GAR}
{FGAR}


In [13]:
som = SOM.soms[0]
print(som.molecules.intersection(Molecule.molecules))
print(som.identifier)

{CH2FH4}
{CH2FH4}


In [14]:
k = re.findall("[0-9a-zA-Z]+", som.identifier)
print(k)

['CH2FH4']


In [15]:
Molecule.getMolecule(k[0])

CH2FH4

In [16]:
list(som.molecules)[0] == Molecule.getMolecule(k[0])

True

In [17]:
SOM.soms[0]

{CH2FH4}

In [18]:
Molecule.molecules

[CH2FH4, FH4, CHOFH4, GAR, FGAR]

In [19]:
SOM.merge(reaction1)

{CH2FH4, FH4}

In [20]:
print(SOM.soms)

[{CHOFH4}, {GAR}, {FGAR}, {CH2FH4, FH4}]


In [21]:
reaction2

CHOFH4 + GAR -> FGAR + FH4

In [22]:
print(SOM.reduce(reaction2))

-------------------------------------
before reduction: reaction.reactants:  [MoleculeStoichiometry(molecule=CHOFH4, stoichiometry=1.0), MoleculeStoichiometry(molecule=GAR, stoichiometry=1.0)]
after reduction: reactants:  [MoleculeStoichiometry(molecule=CHOFH4, stoichiometry=1.0), MoleculeStoichiometry(molecule=GAR, stoichiometry=1.0)]
before reduction: reaction.products:  [MoleculeStoichiometry(molecule=FGAR, stoichiometry=1.0), MoleculeStoichiometry(molecule=FH4, stoichiometry=1.0)]
after reduction: products:  [MoleculeStoichiometry(molecule=FGAR, stoichiometry=1.0), MoleculeStoichiometry(molecule=FH4, stoichiometry=1.0)]
Therefore: reduced =  False
-------------------------------------
-------------------------------------
before reduction: reaction.reactants:  [MoleculeStoichiometry(molecule=CHOFH4, stoichiometry=1.0), MoleculeStoichiometry(molecule=GAR, stoichiometry=1.0)]
after reduction: reactants:  [MoleculeStoichiometry(molecule=GAR, stoichiometry=1.0), MoleculeStoichiometry(m

In [23]:
reaction2.category

'reaction_n_n'

In [24]:
from collections import deque

In [26]:
aa = deque([1,2,3])
aa.popleft()
aa.popleft()

2

In [27]:
SOM.soms

[{CHOFH4}, {GAR}, {FGAR}, {CH2FH4, FH4}]

In [38]:
SOM.soms

[{CHOFH4}, {GAR}, {FGAR}, {CH2FH4, FH4}]

In [40]:
som = SOM.soms[3]
reactants_in = deque([mole_tuple for mole_tuple in  
                            reaction2.reactants if 
                            mole_tuple.molecule in som.molecules])
reactants_out = [mole_tuple for mole_tuple in  
                            reaction2.reactants if 
                            mole_tuple.molecule not in som.molecules]
products_in = deque([mole_tuple for mole_tuple in  
                            reaction2.products if 
                            mole_tuple.molecule in som.molecules])
products_out = [mole_tuple for mole_tuple in  
                            reaction2.products if 
                            mole_tuple.molecule not in som.molecules]

In [45]:
reaction2

CHOFH4 + GAR -> FGAR + FH4

In [41]:
reactants_in

deque([])

In [42]:
reactants_out

[MoleculeStoichiometry(molecule=CHOFH4, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=GAR, stoichiometry=1.0)]

In [43]:
products_in

deque([MoleculeStoichiometry(molecule=FH4, stoichiometry=1.0)])

In [44]:
products_out

[MoleculeStoichiometry(molecule=FGAR, stoichiometry=1.0)]

In [31]:
reaction1`

CH2FH4 -> FH4

In [51]:
a = SOM.soms
print(a)

[{CHOFH4}, {GAR}, {FGAR}, {CH2FH4, FH4}]


In [ ]:
"""
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir))
print(parent_dir)
os.chdir(parent_dir)
print("new directory:", os.getcwd())
"""

In [52]:
a.pop()

{CH2FH4, FH4}

In [53]:
a

[{CHOFH4}, {GAR}, {FGAR}]

In [54]:
import init

ModuleNotFoundError: No module named 'init'